In [10]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.ndimage import gaussian_filter1d

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Задание №1

In [9]:
df_in = pd.read_csv('../data/data_task4_old.txt', sep = '\t', parse_dates = ['assigned_ts','closed_ts'])
def data_prep(df):
    df['delta_dt'] = (df['closed_ts'] - df['assigned_ts'])
    df['delta'] = df['delta_dt'] / pd.Timedelta('1 second')
    df['delta_for_one'] = round(df['delta'] / df['Microtasks'], 0)
    df['delta_for_one_dt'] = (df['delta_dt'] / df['Microtasks']).dt.round('S')
    return df
df = data_prep(df_in)
# Исключаем данные в кторых время выполнения оказалось меньше 0 
df = df[df['delta'] > 0]
top_quantile = .9998 # Явные выбросы
df = df[df['delta_for_one'] < df['delta_for_one'].quantile(top_quantile)] # Удаляем выбросы
delt = df.loc[: , 'delta_for_one']
x_max = 600
alfa = 3
hist_1 = gaussian_filter1d(np.histogram(delt[df['Microtasks'] == 1], bins = range(x_max))[0], alfa)
round_step = 5 
fair_delta = round_step * math.ceil(np.argmax(hist_1)/round_step)
print(round(fair_delta / 30, 1), '* N')

1.2 * N


# Задание №2

In [17]:
df_in = pd.read_csv('../data/data_task3.csv', 
                    sep = '\t', 
                   )
df_in['quality'] = (df_in['jud'] == df_in['cjud']).astype(int)
df_docid = df_in[['docid', 'quality']].groupby(['docid']).mean().sort_values('quality')
docid_list = df_docid[(df_docid['quality'] > 0)&(df_docid['quality'] < 1)].index
def assessors_rating(df):
    df_users = pd.DataFrame(index = df.uid.sort_values().unique())
    for uid in df_users.index:
        jud = df.loc[df['uid'] == uid, 'jud'] # Оценка асессора
        cjud = df.loc[df['uid'] == uid, 'cjud'] # Верная оценка
        df_users.loc[uid, 'accuracy_score'] = accuracy_score(jud, cjud) # Простой процент правильных ответов
        df_users.loc[uid, 'f1_score'] = f1_score(jud, cjud) # f1 метрика полнота + качество
        df_users.loc[uid, 'Count'] = df.loc[df['uid'] == uid, 'uid'].count()
    #     break
    return df_users.sort_values('f1_score', ascending = False)
df_users = assessors_rating(df_in[df_in['docid'].isin(docid_list)])
print('Ответы приведены в порядке ухудшения качества работы ассеора\n')
print('Лучшие 10 асессоров: ', list(df_users.index[:10]))
print('Худшие 10 асессоров: ', list(df_users.index[-10:]))

Ответы приведены в порядке ухудшения качества работы ассеора

Лучшие 10 асессоров:  [487, 596, 171, 184, 259, 545, 315, 217, 166, 570]
Худшие 10 асессоров:  [157, 354, 112, 234, 390, 3, 56, 111, 118, 163]
